In [63]:
import pandas as pd
import numpy as np

arrests = pd.read_csv('arrests.csv')
pop = pd.read_csv('total-population.csv')[['Neighborhood','Estimate; Total']].groupby('Neighborhood').sum()
popdict=pop.to_dict(orient='series')
print(popdict)
arrests=arrests.groupby('INCIDENTNEIGHBORHOOD').size()
arrestsdict=arrests.to_dict()

#print(arrestsdict)

{'Estimate; Total': Neighborhood
Allegheny Center       1411.0
Allegheny West          343.0
Allentown              2558.0
Arlington              1852.0
Arlington Heights       272.0
                        ...  
Upper Lawrenceville    2754.0
West End                235.0
West Oakland           1717.0
Westwood               3537.0
Windgap                1534.0
Name: Estimate; Total, Length: 91, dtype: float64}
